<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h1 style='margin:10px 5px'> 
Master Thesis Yannik Haller - Data Preprocessing VADER
</h1>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
1. Load required packages and the data
</h2>
</div>

In [1]:
# Import required baseline packages
import re
import os
import glob
import time
import sys
import pandas as pd
import numpy as np
from pprint import pprint

# Change pandas' setting to print out long strings
pd.options.display.max_colwidth = 200

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Spacy (for lemmatization)
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim (optional)
import logging
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.ERROR)

import warnings
warnings.filterwarnings("ignore", category = DeprecationWarning)
warnings.filterwarnings("ignore", category = FutureWarning)

C:\Users\Hallk\AppData\Roaming\Python\Python37\site-packages\ipykernel\pylab\config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
# Set the appropriate working directory
os.chdir('D:\\Dropbox\\MA_data')

In [3]:
# Read in the aggregated data
it_tx = pd.read_csv("agg_csv_sparse_it.csv", index_col = 0, dtype = {'so': object, 'la': object, 'tx': object})

In [4]:
# Take a look at the shape of the data
it_tx.shape

(23621, 3)

In [5]:
# Store the article IDs (i.e. index) of the language specific subsets
it_idx = it_tx.index  # Italian

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2. Preprocess the text data
</h2>
</div>

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2.1 Define all required functions to preprocess the data
</h2>
</div>

In [6]:
# Define a function to prepare/pre-clean the text data for the sentiment analysis using the VADER lexicon
def pre_clean_vader(articles):
    # Raise an error if an inappropriate data type is given as an input
    if(not isinstance(articles, list)):
        raise ValueError("Invalid input type. Expected a list.")

    # Keep track of the processing time
    t = time.time()
    # Replace punctuations which are not followed by a blank with punctuations followed by a blank
    articles = [re.sub(r'[\.]', '. ', x) for x in articles]
    # Separate words in which a lowercase letter is followed by a capital letter, since they usually do not belong together
    articles = [re.sub('(^[a-z]*)+([A-Z])', r'\1 \2', x) for x in articles]
    # Correct manually for those cases where a name like 'McDonalds' was separated to Mc Donalds
    articles = [re.sub('Mc ', 'Mc', x) for x in articles]
    # Replace quotation marks with a blank
    articles = [re.sub('«', ' ', x) for x in articles]
    articles = [re.sub('»', ' ', x) for x in articles]
    # Remove percentage signs
    articles = [re.sub('%', ' ', x) for x in articles]
    # Remove distracting hyphens
    articles = [re.sub("-", " ", x) for x in articles]
    articles = [re.sub("–", " ", x) for x in articles]
    # Remove any blank that precedes a comma
    articles = [re.sub(" ,", ",", x) for x in articles]
    # Remove any blank that precedes a dot
    articles = [re.sub(" \.", ".", x) for x in articles]

    # Remove any links starting with http:// or https://
    articles = [re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub('',x) for x in articles]
    # Remove any links starting with www.
    articles = [re.compile('www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub('',x) for x in articles]
    
    # Replace new line characters (i.e. \n) and multiple blanks with a single blank
    articles = [re.sub('\s+', ' ', x) for x in articles]
    # Print out the processing time
    print("Processing time for pre-cleaning: ", str((time.time() - t)/60), "minutes")

    # Return the pre-cleaned text data
    return articles

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
2.2 Apply the preprocessing
</h2>
</div>

In [7]:
# Apply the preprocessing by means of the previously defined function
it_tx = pre_clean_vader(it_tx.tx.values.tolist())

Processing time for pre-cleaning:  0.07454106012980143 minutes


In [8]:
# Take a look at the first element of the preprocessed data
it_tx[0]

' Fermati obbligato a meditare. Hai oppresso il mondo in ginocchio, forse per pregare …Applaudivamo allo stadio. Ora acclamiamo dal balcone, dai ricchi del pallone a splendidi medici. Ci hai tolto la stretta di mano, la cosa più amicale, il bacio. Senza contatti un solo sfiorarsi, come un soffio di vento. Soli come un solco senza seme. A chiederci i mille perché. Convinti che la vita è breve, non va perso neanche un istante, potrebbe nascere la luna piena, in tutta la sua bellezza e verità. Ci torna felice, compiuto il canto: non camminerete mai da soli. Dubitiamo negli esperti del nulla. Lasciamo volare il pipistrello, maestro della biodiversità, senza colpe. Usciamo dall’inferno che toglie il respiro. Reagiamo al torrente della vita. Dissetiamoci alle fonti naturali, sempre convinti che l’amore ci salverà, unica risposta e tutto vince. Felicità, serenità e si tornerà a sorridere. Sicuri che Dio sconfiggerà anche il coronavirus. Rodolfo Fasani, Mesocco'

In [9]:
# Compare the fully preprocessed data to the initial text

'Fermati obbligato a meditare. Hai oppresso il mondo in ginocchio, forse per pregare …Applaudivamo allo stadio. Ora acclamiamo dal balcone, dai ricchi del pallone a splendidi medici.Ci hai tolto la stretta di mano, la cosa più amicale, il bacio. Senza contatti un solo sfiorarsi, come un soffio di vento. Soli come un solco senza seme.A chiederci i mille perché.Convinti che la vita è breve, non va perso neanche un istante, potrebbe nascere la luna piena, in tutta la sua bellezza e verità.Ci torna felice, compiuto il canto: non camminerete mai da soli.Dubitiamo negli esperti del nulla. Lasciamo volare il pipistrello, maestro della biodiversità, senza colpe.Usciamo dall’inferno che toglie il respiro. Reagiamo al torrente della vita. Dissetiamoci alle fonti naturali, sempre convinti che l’amore ci  salverà, unica risposta e tutto vince.Felicità, serenità e si tornerà a  sorridere.Sicuri che Dio sconfiggerà anche il coronavirus.Rodolfo Fasani, Mesocco'

In [10]:
# Create a correctly indexed dataframe containing the preprocessed fulltext data in a column and export it as a csv file
pd.DataFrame(it_tx, index = it_idx, columns = ['tx']).to_csv("Preprocessed/Sentiment_Analysis/Fulltext/it_fulltext_senti.csv", index = True, encoding = 'utf-8-sig')

<div class="alert alert-info" style="background-color:#5d3a8e; color:white; padding:0px 10px; border-radius:5px;"><h2 style='margin:10px 5px'> 
3. Read in and inspect the preprocessed fulltext data
</h2>
</div>

In [11]:
# Define a function to read in the preprocessed fulltext data
def read_preprocessed_fulltext(language):
    # Raise an error if an inadmissible language is chosen
    allowed_languages = ['de', 'en', 'fr', 'it']
    if language not in allowed_languages:
        raise ValueError("Invalid language. Expected one of: %s" % allowed_languages)
    
    # Set the appropriate working directory
    os.chdir('D:\\Dropbox\\MA_data')

    # Define the name of the file to load
    filename = "Preprocessed/Sentiment_Analysis/Fulltext/"+language+"_fulltext_senti.csv"

    # Read in the data
    tx_ft = pd.read_csv(filename, index_col = 0, dtype = {'tx': object})

    # Get the articles' index together with an enumeration to identify their position in the list of precleaned articles
    idx = tx_ft.index
    idx = pd.DataFrame(idx, columns = [language+'_idx'])

    # Extract the text data as a list of articles
    tx_ft = tx_ft.tx.to_list()

    # Return the preprocessed data
    return tx_ft, idx

In [12]:
# Read in the preprocessed fulltext data
it_tx_ft, it_idx = read_preprocessed_fulltext('it')

In [13]:
# Take a look at the first element of the preprocessed fulltext data
it_tx_ft[0]

' Fermati obbligato a meditare. Hai oppresso il mondo in ginocchio, forse per pregare …Applaudivamo allo stadio. Ora acclamiamo dal balcone, dai ricchi del pallone a splendidi medici. Ci hai tolto la stretta di mano, la cosa più amicale, il bacio. Senza contatti un solo sfiorarsi, come un soffio di vento. Soli come un solco senza seme. A chiederci i mille perché. Convinti che la vita è breve, non va perso neanche un istante, potrebbe nascere la luna piena, in tutta la sua bellezza e verità. Ci torna felice, compiuto il canto: non camminerete mai da soli. Dubitiamo negli esperti del nulla. Lasciamo volare il pipistrello, maestro della biodiversità, senza colpe. Usciamo dall’inferno che toglie il respiro. Reagiamo al torrente della vita. Dissetiamoci alle fonti naturali, sempre convinti che l’amore ci salverà, unica risposta e tutto vince. Felicità, serenità e si tornerà a sorridere. Sicuri che Dio sconfiggerà anche il coronavirus. Rodolfo Fasani, Mesocco'

In [14]:
# Take a look at the dataframe containing the according index
it_idx

,it_idx
0,313578
1,460527
2,460528
3,460529
4,460530
...,...
23616,2425111
23617,2425112
23618,2425113
23619,2425114


In [15]:
# Remove unnecessary variables to save RAM
del it_tx_ft, it_idx